<a href="https://colab.research.google.com/github/DarkZagi5015/NLP/blob/main/Salinan_Model_FIX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Mount Drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Install Library

In [ ]:
!pip install -q transformers datasets seqeval evaluate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 24.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you h

In [ ]:
!pip install -q sympy --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 64.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-runtime-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-runtime-cu12 12.5.82 which is incompati

### Import Library


In [ ]:
# ============
# 1. Import Lib
# ============
import json
import nltk
import torch
import numpy as np
from torch.utils.data import Dataset
from transformers import BertTokenizerFast, BertForTokenClassification, Trainer, TrainingArguments
from nltk.tokenize import word_tokenize
import evaluate

# Unduh resource NLTK
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

### Load & BIO Convert


In [ ]:
# =====================
# 2. Load & BIO Convert
# =====================
def read_jsonl(file_path):
    with open(file_path, 'r') as f:
        return [json.loads(line) for line in f]

def ner_to_bio(data):
    dataset = []
    for item in data:
        tokens = word_tokenize(item["data"])
        tags = ["O"] * len(tokens)
        for ent in item["ner_label"]:
            label, _, _, _, start_token, end_token, _ = ent
            for i in range(start_token, end_token):
                if i < len(tokens):
                    tags[i] = f"B-{label}" if i == start_token else f"I-{label}"
        dataset.append((tokens, tags))
    return dataset

# Path ke file
train_path = "/content/drive/MyDrive/NLP/FindVehicle_Official/FindVehicle_train.jsonl"
test_path = "/content/drive/MyDrive/NLP/FindVehicle_Official/FindVehicle_test.jsonl"

# Load dan konversi data
train_data = ner_to_bio(read_jsonl(train_path))
test_data = ner_to_bio(read_jsonl(test_path))

# Buat mapping label
unique_labels = sorted(set(label for _, labels in train_data for label in labels))
label2id = {label: idx for idx, label in enumerate(unique_labels)}
id2label = {v: k for k, v in label2id.items()}

### Dataset Class & Tokenizer


In [ ]:
# ==========================
# 3. Dataset Class & Tokenizer
# ==========================
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

class NERDataset(Dataset):
    def __init__(self, data, tokenizer, label2id, max_len=128):
        self.data = data
        self.tokenizer = tokenizer
        self.label2id = label2id
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        words, labels = self.data[idx]
        tokenized = self.tokenizer(words, is_split_into_words=True, truncation=True, padding='max_length', max_length=self.max_len, return_tensors='pt')
        word_ids = tokenized.word_ids()

        label_ids = []
        previous_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label2id[labels[word_idx]])
            else:
                label_ids.append(label2id[labels[word_idx]] if labels[word_idx].startswith("I-") else -100)
            previous_word_idx = word_idx

        return {
            'input_ids': tokenized['input_ids'].squeeze(),
            'attention_mask': tokenized['attention_mask'].squeeze(),
            'labels': torch.tensor(label_ids)
        }

train_dataset = NERDataset(train_data, tokenizer, label2id)
test_dataset = NERDataset(test_data, tokenizer, label2id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

### Model & Trainer


In [ ]:
# ====================
# 4. Model & Trainer
# ====================
model = BertForTokenClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)

# Load metrik evaluasi
metric = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_preds = [
        [id2label[p] for (p, l) in zip(pred_row, label_row) if l != -100]
        for pred_row, label_row in zip(predictions, labels)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(pred_row, label_row) if l != -100]
        for pred_row, label_row in zip(predictions, labels)
    ]
    return metric.compute(predictions=true_preds, references=true_labels)

training_args = TrainingArguments(
    output_dir="./results",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to=[]  # Disable wandb/logging
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<ipython-input-7-af0893f5e170>:39: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


### Fine-tuning

In [ ]:
# ================
# 5. Fine-tuning
# ================
trainer.train()

Step,Training Loss
500,0.712600
1000,0.475200
1500,0.439700
2000,0.417100
2500,0.404600
3000,0.378500
3500,0.366600
4000,0.352900
4500,0.349900
5000,0.345300


Step,Training Loss
500,0.712600
1000,0.475200
1500,0.439700
2000,0.417100
2500,0.404600
3000,0.378500
3500,0.366600
4000,0.352900
4500,0.349900
5000,0.345300


TrainOutput(global_step=8088, training_loss=0.3833977435627984, metrics={'train_runtime': 1758.1144, 'train_samples_per_second': 36.798, 'train_steps_per_second': 4.6, 'total_flos': 4227714961585920.0, 'train_loss': 0.3833977435627984, 'epoch': 3.0})

### Evaluation


In [ ]:
# ================
# 6. Evaluation
# ================
results = trainer.evaluate()

# Tampilkan hasil evaluasi
print("Hasil Evaluasi:")
for key, value in results.items():
    if isinstance(value, (int, float)):
        print(f"{key}: {value:.4f}")
    elif isinstance(value, dict):
        print(f"{key}:")
        for subkey, subvalue in value.items():
            if isinstance(subvalue, (int, float)):
                print(f"  {subkey}: {subvalue:.4f}")
            else:
                print(f"  {subkey}: {subvalue}")
    else:
        print(f"{key}: {value}")

Hasil Evaluasi:
eval_loss: 0.3068
eval_vehicle_brand:
  precision: 0.4518
  recall: 0.3821
  f1: 0.4140
  number: 17471.0000
eval_vehicle_color:
  precision: 0.5149
  recall: 0.9271
  f1: 0.6621
  number: 18809.0000
eval_vehicle_location:
  precision: 0.5248
  recall: 0.9686
  f1: 0.6807
  number: 9287.0000
eval_vehicle_model:
  precision: 0.2735
  recall: 0.3651
  f1: 0.3127
  number: 17471.0000
eval_vehicle_orientation:
  precision: 0.5094
  recall: 0.9421
  f1: 0.6613
  number: 8187.0000
eval_vehicle_range:
  precision: 1.0000
  recall: 1.0000
  f1: 1.0000
  number: 216.0000
eval_vehicle_type:
  precision: 0.9993
  recall: 1.0000
  f1: 0.9996
  number: 1410.0000
eval_vehicle_type-bus:
  precision: 0.1817
  recall: 0.2444
  f1: 0.2084
  number: 446.0000
eval_vehicle_type-coupe:
  precision: 0.2474
  recall: 0.3778
  f1: 0.2990
  number: 929.0000
eval_vehicle_type-estate_car:
  precision: 0.1552
  recall: 0.2655
  f1: 0.1959
  number: 467.0000
eval_vehicle_type-hatchback:
  precision:

In [ ]:
# ===========================
# 6 Simpan Model & Tokenizer
# ===========================
save_directory = "/content/drive/MyDrive/NLP/FindVehicle_Official/bert-ner-finetuned"

# Simpan model dan tokenizer
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)


('/content/drive/MyDrive/NLP/FindVehicle_Official/bert-ner-finetuned/tokenizer_config.json',
 '/content/drive/MyDrive/NLP/FindVehicle_Official/bert-ner-finetuned/special_tokens_map.json',
 '/content/drive/MyDrive/NLP/FindVehicle_Official/bert-ner-finetuned/vocab.txt',
 '/content/drive/MyDrive/NLP/FindVehicle_Official/bert-ner-finetuned/added_tokens.json',
 '/content/drive/MyDrive/NLP/FindVehicle_Official/bert-ner-finetuned/tokenizer.json')